
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


## Método de resolución de las raíces

Para encontrar las raíces del polinomio, primero se factorizo la expresión de la siguiente manera:

\begin{equation}
f(x)=(x-1)(A²x²-2A+1)
\end{equation}

De lo anterior podemos encontrar la primera raíz de la expresión que seria:
\begin{equation}
x_1=1
\end{equation}

Lo que queda del polinomio es una ecuación cuadrática, por lo que podemos resolverla, pero debemos modificar las raíces de tal manera de evitar perdida de información o importancia, esto se logra eliminando la mayor cantidad de restas posibles.


\begin{equation}
x_2=1+\frac{\sqrt{(A²-1)}}{A}
\end{equation}

\begin{equation}
x_3=\frac{1}{(A²+A\sqrt{(A²-1))}}
\end{equation}

Con las raices anteriores se eliminaron la mayor cantidad de restas posibles, la única no posible es la correspondiente a la que se encuentra dentro de la raíz.

In [59]:
#funcion para pasar a binario

from bitstring import *
import numpy as np

def float_bin(number):
    return (BitArray(float = number, length = 64).bin)


#parametro string, retorna int
def binario_int(binario):
    entero = 0
    cont = 0
    for i in binario:
        entero = entero + (int(i) * (2**(len(binario)-cont-1)))
        cont += 1
    return entero

def int_binario(numero, bits):
    aux = "{0:b}".format(numero) #aux es el numero binario en string
    if (len(aux) < bits):
        cont = bits - len(aux)
        while (cont > 0):
            aux = "0" + aux
            cont-=1
    return aux


    
def exp_en_nuevo_exp(exp_ant, bits_nuevo_exp):
    #se sabe que el exponente antiguo es de 64 bits
    #sacar el rango del nuevo exponente y ver si el numero se puede representar
    cant_num_nuevo = 2**bits_nuevo_exp
    sesgo = (2**(bits_nuevo_exp-1))-1
    limite_inf = 1-sesgo
    limite_sup = cant_num_nuevo-2-sesgo
    #sacar 2^a la cuanto es el exp_ant
    #pasar exp_ant a numero entero y restar su sesgo asi vemos si entra en el nuevo intervalo
    numero_exponente = binario_int(exp_ant)
    sesgo_ant = (2**(len(exp_ant)-1))-1
    if (numero_exponente - sesgo_ant <= limite_sup and numero_exponente - sesgo_ant >= limite_inf):
        exponente_nuevo = numero_exponente-sesgo_ant
        nuevo_exp = int_binario((sesgo+exponente_nuevo), bits_nuevo_exp)
        return nuevo_exp
    elif (numero_exponente - sesgo_ant > limite_sup):  #si se pasa para arriba +- inf
        cont = 0
        nuevo_exp = ""
        while(cont<bits_nuevo_exp):
            nuevo_exp = "1" + nuevo_exp
            cont += 1
        return nuevo_exp
    else:  #se pasa para abajo, se devuelve un cero 
        cont = 0
        nuevo_exp = ""
        while(cont<bits_nuevo_exp):  #puros ceros
            nuevo_exp = "0" + nuevo_exp
            cont += 1
        return nuevo_exp
        
    
def sumar_uno(mantisa):  #mantisa string y mantisa_nueva string. Puede crecer la mantisa
    segundo = "1"
    segundo = segundo.zfill(len(mantisa))
    mantisa_nueva = ''
    extra = 0
    for i in range(len(mantisa) - 1, -1, -1):
        r = extra
        r += 1 if mantisa[i] == '1' else 0
        r += 1 if segundo[i] == '1' else 0
        mantisa_nueva = ('1' if r % 2 == 1 else '0') + mantisa_nueva
        extra = 0 if r < 2 else 1
    if extra != 0:
        mantisa_nueva = '1' + mantisa_nueva
    return mantisa_nueva
   
    
    
    #mantisa en string, bits_nuevos int
def redondear(mantisa, bits_nuevos):
    if (len(mantisa) == bits_nuevos):
        return mantisa
    #ultimo bit cero
    mantisa = mantisa + "0"
    if (mantisa[bits_nuevos]=="0"): #53 cero
        mantisa_nueva = mantisa[:bits_nuevos]#bien  |000000--
        return mantisa_nueva
    if (mantisa[bits_nuevos] == "1"):#53
        if ("1" in mantisa[bits_nuevos+1:]):   #hay un uno pasado el 53
            mantisa_nueva = sumar_uno(mantisa[:bits_nuevos])#bien, |1 -1 - 
            return mantisa_nueva
        if (mantisa[bits_nuevos-1]=="0"):
            mantisa_nueva = mantisa[:bits_nuevos]#bien  0|100000--
            return mantisa_nueva
        else:
            mantisa_nueva = sumar_uno(mantisa[:bits_nuevos])#bien, 1|10000
            return mantisa_nueva
    
def binario_decimal(mantisa):
    decimal = 0
    cont = -1
    for i in mantisa:
        decimal = decimal + (int(i) * (2**(cont)))
        cont = cont - 1
    return decimal


def ieee_double(signo, exp, mantisa):
    numero_exponente = binario_int(exp)
    sesgo = (2**(len(exp)-1))-1
    correr_coma = (numero_exponente- sesgo)
    if ("1" not in exp and "1" not in mantisa): #es un cero
        entero = float(binario_int((mantisa[:correr_coma])))
        decimal = float(binario_decimal(mantisa[correr_coma:]))
        double = entero + decimal
        return double
    if (correr_coma == 0):
        entero = float(binario_int("1"))
        decimal = float(binario_decimal(mantisa))
        double = entero + decimal
    elif (correr_coma < 0): #puros decimales de mantisa
        #agregar ceros de ser necesarios para mas decimales
        cont = 1
        mantisa = "1" + mantisa 
        while(cont<(correr_coma*(-1))):  #puros ceros
            mantisa = "0" + mantisa
            cont += 1
        #pasar mantisa a decimal
        double = binario_decimal(mantisa)
    elif((correr_coma>0) and (correr_coma<=len(mantisa))):
        #pasar a entero los antes de coma
        entero = float(binario_int(("1"+mantisa[:correr_coma])))
        decimal = float(binario_decimal(mantisa[correr_coma:]))
        double = entero + decimal
    else: #agregar ceros a la derecha de mantisa para numeros enteros mas grandes
        cant_agregar = correr_coma - len(mantisa)
        cont = 0
        while(cont<(cant_agregar)):  #puros ceros
            mantisa = mantisa +"0"
            cont += 1
        mantisa = "1" + mantisa
        double = binario_int(mantisa)
    if (signo == "1"):
            double = double*(-1)
    return double

"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):    #funciona con decimales, no e probado cuando el exp no puede
    # Algoritmo de representación de punto flotante modificada.
    x_bin = float_bin(x)  #numero en double
    exp = x_bin[1:12]
    exp = exp_en_nuevo_exp(exp, bits_exp)
    if (("0" not in exp) or ("1" not in exp)): #infinito o cero 
        mantisa_nueva = "0"
        mantisa_nueva = mantisa_nueva.zfill(bits_mant)
        result = ieee_double(x_bin[0], exp, mantisa_nueva)  #se pasa a double
        return  result
    mantisa_nueva = redondear(x_bin[12:], bits_mant)
    if (len(mantisa_nueva)> bits_mant):  #en caso de overflow
        mantisa_nueva = mantisa_nueva[1:]
    result = ieee_double(x_bin[0], exp, mantisa_nueva)  
    return  result

                             
                        
"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1, bits_mant, bits_exp)
    n2 = f_new_rep(n2, bits_mant, bits_exp)
    result = n1 + n2
    result = f_new_rep(result, bits_mant, bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1, bits_mant, bits_exp)
    n2 = f_new_rep(n2, bits_mant, bits_exp)
    result = n1 * n2
    result = f_new_rep(result, bits_mant, bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1, bits_mant, bits_exp)
    n2 = f_new_rep(n2, bits_mant, bits_exp)
    result = n1 / n2
    result = f_new_rep(result, bits_mant, bits_exp)
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    #calculamos la segunda raíz  x_2=1+\frac{\sqrt{(A²-1)}}{A}    si division por cero devulve nan
    root_dos = f_multiplicacion(A, A, bits_mant, bits_exp)
    root_dos = f_suma(root_dos, -1, bits_mant, bits_exp)
    root_dos = (root_dos)**0.5
    root_dos = f_division(root_dos, A, bits_mant, bits_exp)
    root_dos = f_suma(1, root_dos, bits_mant, bits_exp)
    #calculamos la tercera raíz  x_3=\frac{1}{(A²+A\sqrt{(A²-1))}}
    root_tres = f_multiplicacion(A, A, bits_mant, bits_exp)
    root_tres = f_suma(root_tres, -1, bits_mant, bits_exp)
    root_tres = (root_tres)**0.5
    root_tres = f_multiplicacion(A, root_tres, bits_mant, bits_exp)
    root_tres = f_suma(f_multiplicacion(A, A, bits_mant, bits_exp),root_tres, bits_mant, bits_exp)
    if (root_tres == 0):
        root_tres = np.NINF
    else:
        root_tres = f_division(1, root_tres, bits_mant, bits_exp)
    x_roots = [1, root_dos, root_tres]
    x_roots.sort()
    return x_roots
                                 



### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


## Resolución del problema

Para la resolución del problema de encontrar los valóres donde la temperatura es cero, se utiliza el método de la bisección, esto se realizó para ambos lados del día 25, así después se comparan los valores de las raíces encontradas y se queda con la más cercana a 25. Para definir los initial guesst de la funcíón para cada caso, se compara el signo que entrega la función en el día 25 con un posible guesst, sí este posible valor tiene signo contyrario se utiliza en la función. 

In [58]:
import numpy as np



"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""

def evaluar(punto, alpha, beta):
    return (alpha*np.cos(punto)+beta*np.sin(np.log(punto)))


def bisec(limite_in, limite_fin, tol, alpha, beta):
    while ((limite_fin-limite_in)/2 > tol):
        c = ((limite_fin+limite_in)/2)
        if ((evaluar(c, alpha, beta))==0):
            break
        if ((evaluar(limite_in, alpha, beta))*(evaluar(c, alpha, beta))<0 ):
            limite_fin = c
        else:
            limite_in = c
    return ((limite_fin+limite_in)/2)

def raiz_de_oreman(alpha, beta, tol = 1e-10):
    # Su implementación.
    #aqui hay que difinir los primeros limites 
    
    valor_base = evaluar(25, alpha, beta)
    itera_izq = 25
    itera_der = 25
    while(np.sign(evaluar(itera_izq, alpha, beta)) == np.sign(valor_base)):
        itera_izq = itera_izq - 0.05
    while(np.sign(evaluar(itera_der, alpha, beta)) == np.sign(valor_base)):
        itera_der = itera_der + 0.05
    raiz_izq = bisec(itera_izq, 25, tol, alpha, beta)
    raiz_der = bisec(25, itera_der, tol, alpha, beta)
    if (raiz_der - 25 < 25 -raiz_izq):
        root = raiz_der
    else:
        root = raiz_izq
    return root




# Referencias

Apunte Computación Científica 2021-1